# **Real-Time Facial Expression Detection using Few-Shot Learning**

Shri Hari S - PES1UG22AM154

Venkat Subramanian - PES1UG22AM188

Vishwanath Sridhar - PES1UG22AM194

Vismaya Vadana - PES1UG22AM195

Kaggle Data Source to Import FER2013 Dataset

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
msambare_fer2013_path = kagglehub.dataset_download('msambare/fer2013')

print('Data source import complete.')


100%|██████████| 60.3M/60.3M [00:00<00:00, 76.9MB/s]

Extracting files...


Data source import complete.


In [ ]:
import os

# List the contents of the downloaded dataset directory
dataset_dir = msambare_fer2013_path
print("Contents of the dataset directory:")
print(os.listdir(dataset_dir))

# If you want to load the images, specify the path to the train and test folders
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

# List the emotion subdirectories in the training and testing sets
print("Train directory contents:")
print(os.listdir(train_dir))

print("Test directory contents:")
print(os.listdir(test_dir))


Contents of the dataset directory:
['train', 'test']
Train directory contents:
['sad', 'angry', 'fear', 'disgust', 'surprise', 'neutral', 'happy']
Test directory contents:
['sad', 'angry', 'fear', 'disgust', 'surprise', 'neutral', 'happy']


**Installing EasyFSL**

The `easyfsl` library simplifies the implementation of few-shot learning pipelines by providing ready-to-use modules for tasks like Prototypical Networks, Matching Networks, and Relation Networks. It includes utilities for dataset management, task sampling, and model evaluation in few-shot learning scenarios.


Import Statements

In [1]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from tqdm import tqdm
from easyfsl.samplers import TaskSampler
from easyfsl.utils import plot_images, sliding_average
import cv2
import torch.nn.functional as F
from PIL import Image
import random
import numpy as np

train_dir = 'train'
test_dir = 'test'

Defining the constants for training the Meta-Learning Model

In [2]:
# Define constants
image_size = 48  # Change this to match the FER2013 image size
N_WAY = 5  # Number of classes in a task
N_SHOT = 10  # Number of images per class in the support set
N_QUERY = 15  # Number of images per class in the query set
N_EVALUATION_TASKS = 100
N_TRAINING_EPISODES = 60000

Data Augmentation and Preprocessing

In [3]:
# First, let's correct the transforms
train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),  # Convert to tensor first
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])  # Then normalize
])

test_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

In [4]:
# Load the datasets
train_set = ImageFolder(root=train_dir, transform=train_transforms)
test_set = ImageFolder(root=test_dir, transform=test_transforms)

Creating Task Samplers for Training and Testing

In [5]:

train_set.get_labels = lambda: [instance[1] for instance in train_set.samples]
train_sampler = TaskSampler(
    train_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_TRAINING_EPISODES
)

test_set.get_labels = lambda: [instance[1] for instance in test_set.samples]
test_sampler = TaskSampler(
    test_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_EVALUATION_TASKS
)

In [6]:
# Create data loaders
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=2,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)

test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=2,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

**Prototypical Networks Architecture**


Implements Prototypical Networks with a backbone for feature extraction and a projection head for dimensionality reduction, computing class prototypes and classification scores based on Euclidean distances.

In [7]:
class PrototypicalNetworks(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks, self).__init__()
        self.backbone = backbone
        self.projection = nn.Sequential(
            nn.Linear(512, 256),  # Assuming ResNet18's output is 512
            nn.ReLU(),
            nn.Linear(256, 128)
        )

    def forward(self, support_images, support_labels, query_images):
        z_support = self.projection(self.backbone(support_images))
        z_query = self.projection(self.backbone(query_images))

        # Infer the number of different classes from the labels of the support set
        n_way = len(torch.unique(support_labels))
        # Prototype i is the mean of all instances of features corresponding to labels == i
        z_proto = torch.cat(
            [z_support[support_labels == label].mean(0).unsqueeze(0) for label in range(n_way)]
        )

        # Compute the euclidean distance from queries to prototypes
        dists = torch.cdist(z_query, z_proto)

        # Transform distances into classification scores
        scores = -dists
        return scores

**Model Initialization**

Initialize the Prototypical Networks model using a ResNet18 backbone pretrained on ImageNet, with the fully connected layer replaced by a flattening operation.

In [8]:
# Initialize the model
convolutional_network = resnet18(weights='IMAGENET1K_V1')
convolutional_network.fc = nn.Flatten()
model = PrototypicalNetworks(convolutional_network)


**Evaluation Function**

Defines a task-based evaluation process for Prototypical Networks, calculating accuracy across multiple tasks using support and query sets.


In [ ]:
# Evaluation function
def evaluate_on_one_task(support_images: torch.Tensor, support_labels: torch.Tensor,
                         query_images: torch.Tensor, query_labels: torch.Tensor) -> (int, int):
    return (
        torch.max(model(support_images.cuda(), support_labels.cuda(), query_images.cuda()).detach(), 1)[1]
        == query_labels.cuda()
    ).sum().item(), len(query_labels)

def evaluate(data_loader: DataLoader):
    total_predictions = 0
    correct_predictions = 0
    model.eval()
    with torch.no_grad():
        for support_images, support_labels, query_images, query_labels, _ in tqdm(data_loader):
            correct, total = evaluate_on_one_task(support_images, support_labels, query_images, query_labels)
            total_predictions += total
            correct_predictions += correct

    print(f"Model tested on {len(data_loader)} tasks. Accuracy: {(100 * correct_predictions / total_predictions):.2f}%")

# Evaluate the model
evaluate(test_loader)

100%|██████████| 100/100 [00:11<00:00,  8.83it/s]

Model tested on 100 tasks. Accuracy: 29.97%


**Training and Validation Loop**

Implements the training loop for Prototypical Networks with early stopping, a cosine annealing learning rate scheduler, and periodic validation for model performance monitoring and checkpoint saving.


In [ ]:
# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def fit(support_images: torch.Tensor, support_labels: torch.Tensor,
        query_images: torch.Tensor, query_labels: torch.Tensor) -> float:
    optimizer.zero_grad()
    classification_scores = model(support_images.cuda(), support_labels.cuda(), query_images.cuda())
    loss = criterion(classification_scores, query_labels.cuda())
    loss.backward()
    optimizer.step()
    return loss.item()


# Now let's set up the complete training loop with validation
# Initialize variables for early stopping
best_acc = 0
patience = 5
patience_counter = 0

# Create scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.0003, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                T_max=N_TRAINING_EPISODES,
                                                eta_min=1e-6)

# Training loop
log_update_frequency = 10
all_loss = []
model.train()

with tqdm(enumerate(train_loader), total=len(train_loader)) as tqdm_train:
    for episode_index, (support_images, support_labels, query_images, query_labels, _) in tqdm_train:
        # Training step
        loss_value = fit(support_images, support_labels, query_images, query_labels)
        all_loss.append(loss_value)

        # Update learning rate
        scheduler.step()

        # Update progress bar
        if episode_index % log_update_frequency == 0:
            tqdm_train.set_postfix(
                loss=sliding_average(all_loss, log_update_frequency),
                lr=scheduler.get_last_lr()[0]
            )

        # Validation step
        if episode_index % 1000 == 0:
            model.eval()
            current_acc = 0
            n_tasks = 0

            # Compute validation accuracy
            with torch.no_grad():
                for val_support_images, val_support_labels, val_query_images, val_query_labels, _ in test_loader:
                    correct, total = evaluate_on_one_task(
                        val_support_images,
                        val_support_labels,
                        val_query_images,
                        val_query_labels
                    )
                    current_acc += correct
                    n_tasks += total

            current_acc = (100 * current_acc / n_tasks)
            print(f"\nValidation accuracy at episode {episode_index}: {current_acc:.2f}%")

            # Early stopping check
            if current_acc > best_acc:
                best_acc = current_acc
                torch.save({
                    'episode': episode_index,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'accuracy': best_acc,
                }, 'best_model.pth')
                patience_counter = 0
                print(f"New best accuracy: {best_acc:.2f}%")
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print("Early stopping triggered!")
                break

            model.train()

# Load best model and evaluate
checkpoint = torch.load('best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"\nFinal evaluation with best model (acc: {checkpoint['accuracy']:.2f}%):")

  0%|          | 0/60000 [00:00<?, ?it/s, loss=1.61, lr=0.0003]


Validation accuracy at episode 0: 27.87%


  0%|          | 2/60000 [00:08<55:42:13,  3.34s/it, loss=1.61, lr=0.0003] 

New best accuracy: 27.87%


  2%|▏         | 1000/60000 [02:36<2:12:20,  7.43it/s, loss=0.997, lr=0.0003]

...

 45%|████▌     | 27003/60000 [1:10:42<8:03:46,  1.14it/s, loss=0.144, lr=0.000174] 


Validation accuracy at episode 27000: 69.29%


 47%|████▋     | 28000/60000 [1:13:17<1:23:45,  6.37it/s, loss=0.137, lr=0.000166]


Validation accuracy at episode 28000: 69.69%
Early stopping triggered!

Final evaluation with best model (acc: 71.29%):



<ipython-input-31-ee19d09b9781>:90: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_model.pth')


In [ ]:
# Evaluate the model
evaluate(test_loader)


100%|██████████| 100/100 [00:08<00:00, 11.93it/s]

Model tested on 100 tasks. Accuracy: 69.23%


**Model Loading and Final Evaluation**

Dynamically sets the device (CPU or CUDA), loads the best model checkpoint, and prepares for final evaluation with the stored accuracy.


In [9]:
# Dynamically set the device to CPU or CUDA if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load best model and evaluate
checkpoint = torch.load('model.pth', map_location=device)  # Map to the appropriate device
model.load_state_dict(checkpoint['model_state_dict'])
print(f"\nFinal evaluation with best model (acc: {checkpoint['accuracy']:.2f}%):")


Final evaluation with best model (acc: 71.29%):


C:\Users\Vishwanath Sridhar\AppData\Local\Temp\ipykernel_20200\3953788035.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('model.pth', map_locati

**Live Face Recognition with Prototypical Networks**

Implements a real-time face recognition system using a webcam, leveraging Prototypical Networks to classify faces based on embeddings and prototypes with smoothed predictions for robust inference.


In [24]:
# Load pre-trained face detector (Haar Cascade or MTCNN)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Preprocessing for the query image (face detection)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ensure resizing to match the input size used during training
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard normalization
])


# Create a balanced support set with a max of 100 images per class
class_indices = {}
for idx, (_, label) in enumerate(train_set.imgs):
    if label not in class_indices:
        class_indices[label] = []
    class_indices[label].append(idx)

# Limit to a maximum of 100 images per class
balanced_indices = []
for label, indices in class_indices.items():
    balanced_indices.extend(random.sample(indices, min(100, len(indices))))

balanced_train_set = Subset(train_set, balanced_indices)
train_loader = DataLoader(balanced_train_set, batch_size=8, shuffle=True)

# Initialize variables to store the support set embeddings and prototypes
support_embeddings = None
prototypes = None

# Start video capture from the webcam
video_capture = cv2.VideoCapture(0)

if not video_capture.isOpened():
    print("Error: Could not access webcam.")
    exit()

# Variable to keep track of predictions over time (smoothing)
previous_predictions = []
prediction_buffer_size = 10

# Smooth prediction using a moving average or majority vote
def smooth_prediction(predictions, buffer_size):
    return np.bincount(predictions[-buffer_size:]).argmax()

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert frame to grayscale for the detector
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # If faces are detected
    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]

        # Convert numpy.ndarray (OpenCV) to PIL.Image
        face_image = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

        # Apply transformation
        face_tensor = transform(face_image).unsqueeze(0).to(device)  # Add batch dimension and move to device

        # Get a small batch of images as the support set (only once per video loop)
        if support_embeddings is None:  # Initialize the support set embeddings and prototypes
            support_images, support_labels = next(iter(train_loader))  # Get the first batch
            support_images, support_labels = support_images.to(device), support_labels.to(device)

            # Extract embeddings for the support set
            with torch.no_grad():
                support_embeddings = model.backbone(support_images)  # Backbone returns the feature embeddings
                support_embeddings = model.projection(support_embeddings)

                # Calculate prototypes for the support set (mean of support embeddings per class)
                prototypes = []
                for label in torch.unique(support_labels):
                    class_embeddings = support_embeddings[support_labels == label]
                    prototypes.append(class_embeddings.mean(0).unsqueeze(0))
                prototypes = torch.cat(prototypes)

        # Forward pass through the model for query image (live face)
        with torch.no_grad():
            query_embedding = model.backbone(face_tensor)
            query_embedding = model.projection(query_embedding)

            # Calculate distances between query embedding and support prototypes (e.g., Euclidean distance)
            distances = F.pairwise_distance(query_embedding, prototypes)
            predicted_class = distances.argmin().item()

            # Append the current prediction to previous_predictions for smoothing
            previous_predictions.append(predicted_class)

            # Use the moving average of the last 10 predictions for smoothing
            smoothed_prediction = smooth_prediction(previous_predictions, prediction_buffer_size)

            # Display the smoothed predicted class on the live video
            cv2.putText(frame, f"Predicted Class: {smoothed_prediction}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Draw rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Video - Face Detection with Model Inference', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()
